# Reading and cleaning NOAA data

This notebook includes initial exploration of the NOAA weather data. Specifically, it was used to:
1. Develop code for getting NOAA data through API
2. Develop code for cleaning NOAA data prior to merge with tripdata.

In [115]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

In [110]:
def get_all_year_data(year):
    offset=0
    found_all = False
    data = []
    while not found_all:
        url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data?"
        url = url + 'datasetid=GHCND&stationid=GHCND:USW00094728&startdate={0}-01-01&enddate={0}-12-31'.format(year)
        url = url + '&limit=1000&offset={}'.format(offset*1000+1)

        # replace 'myToken' with the actual token, below
        headers = {'token': 'TzFmshIEmgvthbzMdJnlOBcxykLlHPDZ'}
        response = requests.get(url, headers = headers)

        print 'Year: {}, Offset:{}, Response: {}'.format(year, offset, response)
        response = response.json()

        if len(response) == 0:
            found_all = True
        else:
            data.extend(response['results'])
            offset += 1
            
    return data

In [111]:
def get_year_dataframe(year):
    json_data = get_all_year_data(year)
    df = json_normalize(json_data)
    df.drop(['attributes','station'], axis=1, inplace=True)
    df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%dT%H:%M:%S')
    df = df.pivot(index='date',columns='datatype',values='value')
    if year % 4 != 0:
        assert df.shape[0] == 365
    else:
        assert df.shape[0] == 366
    return df

In [112]:
def get_all_years_data(year_range):
    first_year = True
    for year in year_range:
        df = get_year_dataframe(year)
        if first_year:
            all_years = df
            first_year = False
        else:
            all_years = all_years.append(df)
            
    return all_years

In [113]:
all_years = get_all_years_data(range(2014,2017))

Year: 2014, Offset:0, Response: <Response [200]>
Year: 2014, Offset:1, Response: <Response [200]>
Year: 2014, Offset:2, Response: <Response [200]>
Year: 2014, Offset:3, Response: <Response [200]>
Year: 2014, Offset:4, Response: <Response [200]>
Year: 2015, Offset:0, Response: <Response [200]>
Year: 2015, Offset:1, Response: <Response [200]>
Year: 2015, Offset:2, Response: <Response [200]>
Year: 2015, Offset:3, Response: <Response [200]>
Year: 2015, Offset:4, Response: <Response [200]>
Year: 2016, Offset:0, Response: <Response [200]>
Year: 2016, Offset:1, Response: <Response [200]>
Year: 2016, Offset:2, Response: <Response [200]>
Year: 2016, Offset:3, Response: <Response [200]>
Year: 2016, Offset:4, Response: <Response [200]>


In [116]:
all_years.head()

datatype,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT04,WT06,WT08
date,,,,,,,,,,,,,,,
2014-01-01,25.0,0.0,0.0,0.0,6.0,-43.0,280.0,300.0,63.0,103.0,NaN,NaN,NaN,NaN,NaN
2014-01-02,55.0,84.0,79.0,0.0,6.0,-77.0,50.0,50.0,94.0,125.0,NaN,NaN,NaN,NaN,NaN
2014-01-03,47.0,74.0,84.0,150.0,-77.0,-127.0,360.0,310.0,94.0,130.0,1.0,NaN,NaN,NaN,NaN
2014-01-04,22.0,0.0,0.0,150.0,-16.0,-132.0,260.0,240.0,40.0,89.0,NaN,NaN,NaN,NaN,NaN
2014-01-05,17.0,36.0,0.0,100.0,44.0,-27.0,70.0,240.0,31.0,76.0,NaN,NaN,NaN,NaN,NaN


In [117]:
all_years.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2014-01-01 to 2016-12-31
Data columns (total 15 columns):
AWND    1083 non-null float64
PRCP    1096 non-null float64
SNOW    1096 non-null float64
SNWD    1096 non-null float64
TMAX    1096 non-null float64
TMIN    1096 non-null float64
WDF2    1086 non-null float64
WDF5    1075 non-null float64
WSF2    1086 non-null float64
WSF5    1075 non-null float64
WT01    312 non-null float64
WT02    14 non-null float64
WT04    4 non-null float64
WT06    6 non-null float64
WT08    251 non-null float64
dtypes: float64(15)
memory usage: 137.0 KB


Based on the number of missing values (and rarity of weather events WT02, WT04, and WT06 which are heavy fog, ice pellets/sleet, and glaze or rime), we will only use the following features:
    - PRCP: Precipitation
    - SNOW: Snowfall
    - SNWD: Snow depth
    - TMAX: Max temperature
    - TMIN: Min temperature
    - AWND: Average daily wind speed
    - WSF2: Fastest 2-minute wind speed
    - WSF5: Fastest 4-second wind speed
    - WT01: Fog, ice fog, or freezing fog (may include heavy fog)
    - WT08: Smoke or haze

In [119]:
all_years = all_years[['PRCP','SNOW','SNWD','TMAX','TMIN','AWND','WSF2','WSF5', 'WT01', 'WT08']]

In [120]:
all_years.head()

datatype,PRCP,SNOW,SNWD,TMAX,TMIN,AWND,WSF2,WSF5,WT01,WT08
date,,,,,,,,,,
2014-01-01,0.0,0.0,0.0,6.0,-43.0,25.0,63.0,103.0,NaN,NaN
2014-01-02,84.0,79.0,0.0,6.0,-77.0,55.0,94.0,125.0,NaN,NaN
2014-01-03,74.0,84.0,150.0,-77.0,-127.0,47.0,94.0,130.0,1.0,NaN
2014-01-04,0.0,0.0,150.0,-16.0,-132.0,22.0,40.0,89.0,NaN,NaN
2014-01-05,36.0,0.0,100.0,44.0,-27.0,17.0,31.0,76.0,NaN,NaN


In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2014-01-01 to 2014-12-31
Data columns (total 12 columns):
AWND    364 non-null float64
PRCP    365 non-null float64
SNOW    365 non-null float64
SNWD    365 non-null float64
TMAX    365 non-null float64
TMIN    365 non-null float64
WDF2    365 non-null float64
WDF5    360 non-null float64
WSF2    365 non-null float64
WSF5    360 non-null float64
WT01    71 non-null float64
WT08    44 non-null float64
dtypes: float64(12)
memory usage: 37.1 KB


Next,we fill the null weather type values with 0's.

In [122]:
all_years = all_years.fillna({'WT01':0, 'WT08':0})

In [123]:
all_years.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1096 entries, 2014-01-01 to 2016-12-31
Data columns (total 10 columns):
PRCP    1096 non-null float64
SNOW    1096 non-null float64
SNWD    1096 non-null float64
TMAX    1096 non-null float64
TMIN    1096 non-null float64
AWND    1083 non-null float64
WSF2    1086 non-null float64
WSF5    1075 non-null float64
WT01    1096 non-null float64
WT08    1096 non-null float64
dtypes: float64(10)
memory usage: 94.2 KB


To finish cleaning the weather data, all we need to do is fill missing wind speed values. We'll do so by averaging the forward and backfill values (under the intuition that wind speed is essentially continuous, so the mean value is a reasonable estimate).

In [124]:
def get_forward_back_avg(series):
    forward = series.ffill()
    back = series.bfill()
    if np.sum(forward - back) == 0:
        print 'No change for {}'.format(series.name)
    average = (forward + back)/2.0
    return average

In [125]:
for col in df.columns:
    df[col] = get_forward_back_avg(df[col])

No change for PRCP
No change for SNOW
No change for SNWD
No change for TMAX
No change for TMIN
No change for WDF2
No change for WSF2
No change for WT01
No change for WT08
